In [6]:
!pip install --upgrade -q gspread

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('google_image_download_data')


In [4]:
# This loads a 64 character api key from a file stored on Google Drive
# If you need to run this, it's just an API key for SerpAPI
import os
import pandas as pd

directory_config = "/content/drive/MyDrive/azure_config/" # Change this to suit your machine

path_config = os.path.join(directory_config,"serp.json")

# Verify the configuration file exists
if(os.path.exists(path_config)):
    print("Success.",path_config, "exists.")
else:
    print("Failure.",path_config, "does not exist.")

# Read the JSON file into a DataFrame
df_config = pd.read_json(path_config)
#print(df_config['COMPUTER_VISION_SUBSCRIPTION_KEY'].iloc[0])
#print(df_config['COMPUTER_VISION_ENDPOINT'].iloc[0])

# Store as enivonmental variables
os.environ['SERP_KEY'] = df_config['SERP_KEY'].iloc[0]


# Do some basic validation
if len(os.environ['SERP_KEY']) == 64:
    print("Success, SERP_KEY is loaded.")
else:
    print("Error, The SERP_KEY is not the expected length, please check it.",len(os.environ['SERP_KEY']) )

Success. /content/drive/MyDrive/azure_config/serp.json exists.
Success, SERP_KEY is loaded.


In [4]:
!pip install google-search-results

  Created wheel for google-search-results: filename=google_search_results-2.4.1-py3-none-any.whl size=25789 sha256=4e491eb55da6639751e8cceb6a6506b48f6b292df608554fc4ce757e6d4d2971
  Stored in directory: /root/.cache/pip/wheels/82/a3/c5/364155118f298722dff2f79ae4dd7c91e92b433ad36d6f7e0e
Successfully built google-search-results


In [12]:
import os, urllib.request, json # json for pretty output
from serpapi import GoogleSearch


def get_google_images(ijn, worksheet_search, search_name, search_term):
    params = {
      "api_key": os.getenv("SERP_KEY"),
      "engine": "google",
      "q": search_term,
      "tbm": "isch",
      "ijn": ijn
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    print(results)
    #print(json.dumps(results['suggested_searches'], indent=2, ensure_ascii=False))
    print(json.dumps(results['images_results'], indent=2, ensure_ascii=False))

    # -----------------------
    # Downloading images

    for index, image in enumerate(results['images_results']):
        image_number = index+(100*ijn)  

        print(f'Downloading {image_number} image...')
        
        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582')]
        urllib.request.install_opener(opener)
        try:
            urllib.request.urlretrieve(image['original'], f'/content/drive/MyDrive/google_image_download_images/{search_name}/original_size_img_{image_number}.jpg')
            worksheet_search.update_cell(image_number+2, 1, image_number)
            worksheet_search.update_cell(image_number+2, 2, f'original_size_img_{image_number}.jpg')
        except Exception as e:
            print("Error", e)  
            worksheet_search.update_cell(image_number+2, 1, image_number)
            worksheet_search.update_cell(image_number+2, 2, f'ERROR {e}')


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('google_image_download_data')

#search_name = "pula_amphitheater"
#search_term = "'pula amphitheater'"

search_name = "pula_temple_augustus"
search_term = "'pula temple augustus'"

worksheet_search = sh.worksheet(title=search_name)
get_google_images(0,worksheet_search, search_name, search_term)




https://serpapi.com/search
{'search_metadata': {'id': '627c554049ecdbea7001b86e', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/e76af82e9fad32f1/627c554049ecdbea7001b86e.json', 'created_at': '2022-05-12 00:30:56 UTC', 'processed_at': '2022-05-12 00:30:56 UTC', 'google_url': 'https://www.google.com/search?q=%27pula+temple+augustus%27&oq=%27pula+temple+augustus%27&tbm=isch&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/e76af82e9fad32f1/627c554049ecdbea7001b86e.html', 'total_time_taken': 0.94}, 'search_parameters': {'engine': 'google', 'q': "'pula temple augustus'", 'google_domain': 'google.com', 'ijn': '0', 'device': 'desktop', 'tbm': 'isch'}, 'search_information': {'image_results_state': 'Results for exact spelling', 'query_displayed': "'pula temple augustus'"}, 'images_results': [{'position': 1, 'thumbnail': 'https://serpapi.com/searches/627c554049ecdbea7001b86e/images/a99dfe78aa21767c3e95737c1a335494719e94e0baa95d4102bb87949b2e7719.jpeg

In [ ]:
#get_google_images(1,worksheet_search, search_name, search_term)
get_google_images(2,worksheet_search, search_name, search_term)
get_google_images(3,worksheet_search, search_name, search_term)
get_google_images(4,worksheet_search, search_name, search_term)

In [10]:
worksheet_search = sh.worksheet(title="pula_amphitheater")

#/content/drive/MyDrive/google_image_download_images/pula_ampitheater